<a href="https://colab.research.google.com/github/danieltalon/trabalho_RAG/blob/main/trabalho_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install langchain langchain-community langchain-google-genai
%pip install pypdf sentence-transformers faiss-cpu tiktoken python-dotenv
print("Bibliotecas instaladas.")

# pra que serve o "-q" no pip install?
# Se for usar LlamaCpp (alternativa ao Gemini):
# !pip install llama-cpp-python -q

  Using cached langchain_community-0.3.22-py3-none-any.whl.metadata (2.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successf

Bibliotecas instaladas.


In [32]:
import os
import time
import warnings
from google.colab import userdata, drive # Para secrets e Google Drive
import torch # Para verificar GPU
import pandas as pd
from IPython.display import display # Para mostrar tabelas

# Langchain e componentes relacionados

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI # Usando Gemini
# from langchain_community.chat_models import ChatOllama # Alternativa Ollama
# from langchain_community.llms import LlamaCpp # Alternativa LlamaCpp

# Ignorar avisos específicos se necessário (opcional)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [33]:
# --- Configurações ---

# 1. Montar Google Drive (Recomendado para persistência)
try:
    drive.mount('/content/drive')
    DRIVE_MOUNTED = True
    # Defina um caminho base no seu Google Drive
    BASE_DRIVE_PATH = "/content/drive/MyDrive/RAG/" # <--- AJUSTE SE NECESSÁRIO
    os.makedirs(BASE_DRIVE_PATH, exist_ok=True)
    print(f"Google Drive montado em /content/drive. Usando base: {BASE_DRIVE_PATH}")
except Exception as e:
    DRIVE_MOUNTED = False
    BASE_DRIVE_PATH = "/content/" # Usar armazenamento local temporário do Colab
    print(f"Google Drive não montado ou erro: {e}. Usando armazenamento local temporário: {BASE_DRIVE_PATH}")
    print("AVISO: O índice vetorial NÃO será salvo permanentemente.")

# 2. Caminhos para os arquivos (ajuste conforme onde você colocou os arquivos)
#    Coloque seu PDF e perguntas.txt na pasta BASE_DRIVE_PATH no seu Google Drive
#    OU faça upload manual para /content/ no Colab (mas serão perdidos ao desconectar)
PDF_PATH = os.path.join(BASE_DRIVE_PATH, "saude_coletiva.pdf")
QUESTIONS_PATH = os.path.join(BASE_DRIVE_PATH, "perguntas.txt")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado em /content/drive. Usando base: /content/drive/MyDrive/RAG/


In [34]:
# --- Definição das Configurações de RAG para Comparar ---

# Configuração A: MiniLM Embeddings, Chunks Maiores, k=4
config_A = {
    "name": "MiniLM_LargeChunks_k4",
    "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
    "chunk_size": 1500,
    "chunk_overlap": 300,
    "k_retrieval": 4,
    "vector_store_path": os.path.join(BASE_DRIVE_PATH, "faiss_index_config_A"),
    "llm_provider": "gemini", # Manter o mesmo LLM para comparar o retrieval/embedding
    "llm_model": "gemini-1.5-flash-latest"
}

# Configuração B: MPNet Embeddings (melhor qualidade), Chunks Menores, k=5
config_B = {
    #"name": "MPNet_SmallChunks_k5",
    #"embedding_model": "sentence-transformers/all-mpnet-base-v2", # Modelo diferente
    "name": "QA-MiniLM_SmallChunks_k5", # Atualize o nome para refletir a mudança
    "embedding_model": "sentence-transformers/multi-qa-MiniLM-L6-cos-v1", # <--- MODELO ALTERADO AQUI
    "chunk_size": 800, # Tamanho diferente
    "chunk_overlap": 150, # Overlap diferente
    "k_retrieval": 5, # k diferente
    "vector_store_path": os.path.join(BASE_DRIVE_PATH, "faiss_index_config_B"), # Caminho DIFERENTE
    "llm_provider": "gemini", # Mesmo LLM
    "llm_model": "gemini-1.5-flash-latest"
}

# Lista de configurações a serem testadas
configurations = [config_A, config_B]

# Carregar API Key do Gemini (apenas uma vez se ambas usarem Gemini)
# Coloque isso fora das configs se for o mesmo para todas
LLM_REQUIRES_API_KEY = any(conf.get("llm_provider") == "gemini" for conf in configurations)
if LLM_REQUIRES_API_KEY:
    try:
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        if not GOOGLE_API_KEY:
            raise ValueError("Secret 'GOOGLE_API_KEY' não encontrado ou vazio.")
        os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
        print("API Key do Google carregada via Colab Secrets.")
    except Exception as e:
        print(f"Erro ao carregar GOOGLE_API_KEY: {e}")
        raise SystemExit("Chave API do Google não configurada.")


# --- Escolha e Configuração do LLM ---
##LLM_PROVIDER = "gemini" # Ou "ollama", "llamacpp"

# Configuração específica do Gemini
#if LLM_PROVIDER == "gemini":
#    try:
        # Use o gerenciador de Secrets do Colab (ícone de chave na barra esquerda)
        # Adicione um secret chamado 'GOOGLE_API_KEY' com sua chave API do Google AI Studio
#        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
#        if not GOOGLE_API_KEY:
#            raise ValueError("Secret 'GOOGLE_API_KEY' não encontrado ou vazio.")
#        os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
#        GEMINI_MODEL = "gemini-1.5-flash-latest" # Modelo Gemini rápido e eficiente
#        print("API Key do Google carregada via Colab Secrets.")
#    except Exception as e:
#        print(f"Erro ao carregar GOOGLE_API_KEY do Colab Secrets: {e}")
#        print("Por favor, adicione sua chave API do Google AI Studio como um Secret chamado 'GOOGLE_API_KEY' no Colab.")
        # Interrompe a execução se a chave não for encontrada
#        raise SystemExit("Chave API do Google não configurada.")

# Configuração alternativa (Ollama - requer Ollama rodando em algum lugar acessível)
# elif LLM_PROVIDER == "ollama":
#     OLLAMA_BASE_URL = "http://localhost:11434" # Se rodando localmente e usando ngrok/proxy para expor ao Colab
#     OLLAMA_MODEL = "mistral" # Modelo rodando no Ollama
#     print(f"Configurado para usar Ollama em {OLLAMA_BASE_URL} com modelo {OLLAMA_MODEL}")

# Configuração alternativa (LlamaCpp - requer download do modelo GGUF)
# elif LLM_PROVIDER == "llamacpp":
#     # Faça upload do seu modelo .gguf para o Drive ou Colab
#     LLAMA_CPP_MODEL_PATH = os.path.join(BASE_DRIVE_PATH, "seu_modelo.gguf") # <--- AJUSTE O NOME DO ARQUIVO
#     print(f"Configurado para usar LlamaCpp com modelo em: {LLAMA_CPP_MODEL_PATH}")
#     if not os.path.exists(LLAMA_CPP_MODEL_PATH):
#         print(f"AVISO: Modelo LlamaCpp não encontrado em {LLAMA_CPP_MODEL_PATH}")
#         # Poderia tentar baixar aqui se tiver URL

API Key do Google carregada via Colab Secrets.


In [35]:
# 4. Verificar disponibilidade de GPU
if torch.cuda.is_available():
    DEVICE = 'cuda'
    print("GPU detectada! Usando CUDA.")
else:
    DEVICE = 'cpu'
    print("GPU não detectada ou não configurada. Usando CPU.")

GPU não detectada ou não configurada. Usando CPU.


In [36]:
def load_pdf_docs(pdf_path):
    """Carrega o documento PDF."""
    if not os.path.exists(pdf_path):
         raise FileNotFoundError(f"Arquivo PDF não encontrado em: {pdf_path}. Verifique o upload ou o caminho no Drive.")
    print(f"Carregando PDF de: {pdf_path}")
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    print(f"PDF carregado. Número de páginas/documentos iniciais: {len(documents)}")
    return documents

def split_docs_into_chunks(docs, chunk_size=1000, chunk_overlap=200):
    """Divide os documentos carregados em chunks menores."""
    print(f"Dividindo documentos em chunks (tamanho: {chunk_size}, sobreposição: {chunk_overlap})")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(docs)
    print(f"Número de chunks criados: {len(chunks)}")
    return chunks

def get_embedding_model(model_name, device):
    """Retorna o modelo de embedding HuggingFace especificado."""
    print(f"Carregando embeddings HuggingFace: {model_name} no dispositivo: {device}")
    try:
        model_kwargs = {'device': device}
        encode_kwargs = {'normalize_embeddings': False}
        embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
        )
        # embeddings.embed_query("Teste rápido") # Opcional: Teste de carregamento
        return embeddings
    except ImportError:
        raise ImportError("Instale 'sentence-transformers': !pip install sentence-transformers")
    except Exception as e:
        print(f"Erro ao carregar modelo de embedding {model_name}: {e}")
        raise

def create_or_load_vector_store(chunks, embeddings, store_path):
    """Cria um novo vector store FAISS ou carrega um existente."""
    if os.path.exists(store_path):
        print(f"Carregando Vector Store FAISS existente de: {store_path}")
        try:
            vector_store = FAISS.load_local(store_path, embeddings, allow_dangerous_deserialization=True)
            print("Vector Store carregado com sucesso.")
        except Exception as e:
            print(f"Erro ao carregar Vector Store: {e}. Tentando recriar...")
            os.remove(store_path) # Remove o índice potencialmente corrompido/incompatível
            vector_store = create_new_vector_store(chunks, embeddings, store_path)
    else:
        vector_store = create_new_vector_store(chunks, embeddings, store_path)
    return vector_store

def create_new_vector_store(chunks, embeddings, store_path):
    """Cria e salva um novo vector store FAISS."""
    print("Criando novo Vector Store FAISS...")
    start_time = time.time()
    vector_store = FAISS.from_documents(chunks, embeddings)
    end_time = time.time()
    print(f"Vector Store criado em {end_time - start_time:.2f} segundos.")
    print(f"Salvando Vector Store em: {store_path}")
    vector_store.save_local(store_path)
    print("Vector Store salvo.")
    return vector_store


def load_questions(questions_path):
    """Carrega as perguntas do arquivo de texto."""
    if not os.path.exists(questions_path):
         raise FileNotFoundError(f"Arquivo de perguntas não encontrado em: {questions_path}. Verifique o upload ou o caminho no Drive.")
    print(f"Carregando perguntas de: {questions_path}")
    try:
        with open(questions_path, 'r', encoding='utf-8') as f:
            questions = [line.strip() for line in f if line.strip()]
        print(f"Carregadas {len(questions)} perguntas.")
        return questions
    except Exception as e:
        print(f"Erro ao ler o arquivo de perguntas '{questions_path}': {e}")
        return []

def get_llm(llm_config):
    """Retorna a instância do LLM com base na configuração fornecida."""
    provider = llm_config.get("llm_provider")
    model = llm_config.get("llm_model")
    print(f"Configurando LLM Provider: {provider} com modelo: {model}")

    if provider == "gemini":
        try:
            # A chave API já deve estar no ambiente via Célula 2
            if 'GOOGLE_API_KEY' not in os.environ:
                 raise ValueError("Chave API do Google não encontrada no ambiente.")
            llm = ChatGoogleGenerativeAI(model=model,
                                         temperature=0.3,
                                         convert_system_message_to_human=True)
            # llm.invoke("Olá!") # Teste rápido
            print(f"Usando Google Gemini: {model}")
            return llm
        except Exception as e:
            print(f"Erro ao instanciar ChatGoogleGenerativeAI: {e}")
            raise

       # Adicione aqui blocos elif para outros providers (Ollama, LlamaCpp) se necessário,
    # buscando os parâmetros relevantes de llm_config
    # elif provider == "ollama":
    #    ollama_model = llm_config.get("ollama_model", "mistral") # Exemplo
    #    ollama_base_url = llm_config.get("ollama_base_url", "http://localhost:11434") # Exemplo
    #    # ... código para instanciar ChatOllama ...
    #    return llm

    else:
        raise ValueError(f"Provedor LLM desconhecido ou não configurado: {provider}")


def setup_qa_chain(llm, vector_store, k_retrieval=5): # Definir k padrão aqui
    """Configura a cadeia de RetrievalQA."""
    print(f"Configurando a cadeia de QA (RetrievalQA) para recuperar k={k_retrieval} chunks...")
    retriever = vector_store.as_retriever(search_kwargs={'k': k_retrieval}) # Usar k_retrieval
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        verbose=False
    )
    print("Cadeia de QA configurada.")
    return qa_chain

def ask_questions(qa_chain, questions, vector_store, k_retrieval=5): # Adicionar vector_store e k_retrieval
    """Itera sobre as perguntas, obtém respostas e calcula score de relevância."""
    print("\n--- Iniciando Sessão de Perguntas e Respostas ---")
    if not questions:
        print("Nenhuma pergunta para processar.")
        return

    results = []
    for i, question in enumerate(questions):
        print(f"\n[Pergunta {i+1}/{len(questions)}]: {question}")
        start_time = time.time()
        avg_relevance_score = 0.0
        retrieved_docs_count = 0

        try:
            # 1. Obter a resposta do LLM via cadeia RAG
            response = qa_chain.invoke({"query": question})
            answer = response.get('result', 'Nenhuma resposta gerada.')
            source_docs = response.get('source_documents', []) # Documentos usados pelo LLM
            end_time = time.time()

            # 2. Calcular o score de relevância médio dos chunks recuperados
            #    Fazemos uma busca separada aqui para obter os scores de relevância normalizados
            #    Idealmente, a cadeia retornaria isso, mas esta é uma forma prática.
            #    Garante que estamos usando o mesmo k que a cadeia de QA.
            if vector_store:
                docs_with_scores = vector_store.similarity_search_with_relevance_scores(
                    question, k=k_retrieval
                )
                retrieved_docs_count = len(docs_with_scores)
                if docs_with_scores:
                    scores = [score for doc, score in docs_with_scores]
                    if scores:
                        avg_relevance_score = sum(scores) / len(scores)

            # 3. Imprimir resultados, incluindo o score
            print(f"[Resposta]: {answer}")
            print(f"(Tempo: {end_time - start_time:.2f}s | Confiança Recuperação (Avg Score): {avg_relevance_score:.4f})")
            print(f"  (Score médio de relevância dos {retrieved_docs_count} chunks recuperados. 1 = Mais Relevante, 0 = Menos Relevante)")

            results.append({
                "pergunta": question,
                "resposta": answer,
                "avg_relevance_score": avg_relevance_score,
                "fontes": source_docs # Mantém as fontes usadas pelo LLM
            })

            # Opcional: Mostrar trechos das fontes usadas (como antes)
            # print("\n[Fontes Utilizadas pelo LLM (Chunks)]:")
            # ... (código para mostrar fontes) ...

        except Exception as e:
            print(f"Erro ao processar a pergunta: {e}")
            results.append({
                "pergunta": question,
                "resposta": f"Erro: {e}",
                "avg_relevance_score": 0.0, # Score 0 em caso de erro
                "fontes": []
            })
        finally:
             print("-" * 50)
    return results


In [37]:
print("Iniciando Comparação de Configurações RAG...")

# Carregar dados comuns uma vez
try:
    base_documents = load_pdf_docs(PDF_PATH)
    questions = load_questions(QUESTIONS_PATH)
    if not base_documents or not questions:
        raise SystemExit("Erro ao carregar PDF ou perguntas. Verifique os arquivos.")
except Exception as e:
    raise SystemExit(f"Erro na carga inicial: {e}")

# Armazenar resultados de todas as configurações
all_results = {}

# Iterar sobre cada configuração definida
for config in configurations:
    config_name = config["name"]
    print(f"\n===== Processando Configuração: {config_name} =====")

    try:
        # 1. Chunking (específico da config)
        chunks = split_docs_into_chunks(base_documents, config["chunk_size"], config["chunk_overlap"])
        if not chunks:
            print(f"AVISO: Nenhum chunk gerado para {config_name}. Pulando esta config.")
            all_results[config_name] = [] # Armazena lista vazia para indicar falha/sem chunks
            continue

        # 2. Embedding (específico da config)
        embeddings = get_embedding_model(config["embedding_model"], DEVICE)

        # 3. Vector Store (específico da config)
        vector_store = create_or_load_vector_store(chunks, embeddings, config["vector_store_path"])

        # 4. LLM (específico da config)
        llm = get_llm(config) # Passa a sub-configuração do LLM

        # 5. QA Chain (específico da config)
        qa_chain = setup_qa_chain(llm, vector_store, k_retrieval=config["k_retrieval"])

        # 6. Executar Perguntas
        config_results = ask_questions(qa_chain, questions, vector_store, k_retrieval=config["k_retrieval"])
        all_results[config_name] = config_results # Armazena os resultados desta config

    except Exception as e:
        print(f"\nERRO ao processar configuração {config_name}")
        print(f"Erro: {e}")
        import traceback
        traceback.print_exc()
        all_results[config_name] = [] # Armazena lista vazia para indicar falha

    print(f"===== Concluída Configuração: {config_name} =====")

##################################


        # Opcional: Salvar resultados em um arquivo (útil no Colab)
        # results_path = os.path.join(BASE_DRIVE_PATH, "rag_results.txt")
        # print(f"\nSalvando resultados em: {results_path}")
        # with open(results_path, "w", encoding="utf-8") as f:
        #     for res in qa_results:
        #         f.write(f"Pergunta: {res['pergunta']}\n")
        #         f.write(f"Resposta: {res['resposta']}\n")
        #         f.write("-" * 20 + "\n")
        # print("Resultados salvos.")


Iniciando Comparação de Configurações RAG...
Carregando PDF de: /content/drive/MyDrive/RAG/saude_coletiva.pdf
PDF carregado. Número de páginas/documentos iniciais: 142
Carregando perguntas de: /content/drive/MyDrive/RAG/perguntas.txt
Carregadas 5 perguntas.

===== Processando Configuração: MiniLM_LargeChunks_k4 =====
Dividindo documentos em chunks (tamanho: 1500, sobreposição: 300)
Número de chunks criados: 307
Carregando embeddings HuggingFace: sentence-transformers/all-MiniLM-L6-v2 no dispositivo: cpu
Carregando Vector Store FAISS existente de: /content/drive/MyDrive/RAG/faiss_index_config_A
Vector Store carregado com sucesso.
Configurando LLM Provider: gemini com modelo: gemini-1.5-flash-latest
Usando Google Gemini: gemini-1.5-flash-latest
Configurando a cadeia de QA (RetrievalQA) para recuperar k=4 chunks...
Cadeia de QA configurada.

--- Iniciando Sessão de Perguntas e Respostas ---

[Pergunta 1/5]: 1. O que é saude coletiva?
[Resposta]: The provided text does not define "saúde co

In [38]:
# --- Geração da Tabela Comparativa ---
print("\n\n===== Tabela Comparativa de Resultados =====")

# Verificar se temos resultados para comparar
if not all_results or len(all_results) < 2:
     print("Não há resultados suficientes para comparação (necessário pelo menos 2 configurações bem-sucedidas).")
else:
    # Estruturar dados para o DataFrame
    comparison_data = []
    num_questions = len(questions)

    for i in range(num_questions):
        row_data = {"Pergunta": questions[i]}
        for config_name, results in all_results.items():
            if i < len(results): # Verifica se há resultado para esta pergunta nesta config
                res = results[i]
                # Usar nomes de coluna mais curtos e claros para CSV
                row_data[f"{config_name}_Resp"] = res.get('resposta', 'N/A')
                row_data[f"{config_name}_T(s)"] = res.get('tempo_s', '-')
                row_data[f"{config_name}_Score"] = res.get('avg_relevance_score', '-')
            else: # Caso de erro ou menos perguntas processadas
                row_data[f"{config_name}_Resp"] = "N/A (Erro ou não processado)"
                row_data[f"{config_name}_T(s)"] = "-"
                row_data[f"{config_name}_Score"] = "-"
        comparison_data.append(row_data)

    # Criar DataFrame
    df_comparison = pd.DataFrame(comparison_data)

    # Configurar Pandas para mostrar mais texto nas colunas (opcional)
    pd.set_option('display.max_colwidth', 150) # Ajuste o valor conforme necessário
    pd.set_option('display.width', 1000) # Ajuste para largura da tela

    # Exibir DataFrame no Colab
    print("\n--- Visualização da Tabela ---")
    display(df_comparison)
    # Se não estiver no Colab ou quiser output em texto puro:
    # print(df_comparison.to_string())

    # --- Salvar DataFrame em Arquivo CSV ---
    output_filename = "rag_comparison_results.csv"
    output_path = os.path.join(BASE_DRIVE_PATH, output_filename)

    try:
        # Salvar em CSV com encoding UTF-8 (bom para caracteres especiais)
        # index=False para não salvar o índice numérico do pandas no arquivo
        df_comparison.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"\n--- Tabela de comparação salva com sucesso em: ---")
        print(output_path)
        if not DRIVE_MOUNTED:
             print("\nAVISO: O arquivo foi salvo no armazenamento temporário do Colab.")
             print("Faça o download manualmente no painel 'Arquivos' antes de fechar a sessão!")
    except Exception as e:
        print(f"\n--- ERRO ao salvar a tabela em arquivo: {e} ---")
        print(f"Verifique as permissões de escrita em: {BASE_DRIVE_PATH}")

print("\nProcesso de Comparação RAG concluído.")



===== Tabela Comparativa de Resultados =====

--- Visualização da Tabela ---


,Pergunta,MiniLM_LargeChunks_k4_Resp,MiniLM_LargeChunks_k4_T(s),MiniLM_LargeChunks_k4_Score,QA-MiniLM_SmallChunks_k5_Resp,QA-MiniLM_SmallChunks_k5_T(s),QA-MiniLM_SmallChunks_k5_Score
0,1. O que é saude coletiva?,"The provided text does not define ""saúde coletiva"". While the text discusses health promotion, the health system in Brazil, and the social determ...",-,0.381385,"Based on the provided text, I cannot answer what ""saúde coletiva"" is. The text mentions it frequently and discusses aspects of it, such as promot...",-,0.553581
1,2. Quais as doenças mais comuns do Brasil?,"I am sorry, but this document does not contain information on the most common diseases in Brazil.",-,0.274078,"I'm sorry, but this text does not list the most common diseases in Brazil. While it mentions some diseases like Hansen's disease and discusses as...",-,0.320044
2,3. Quais as metas propostas pela Organização Mundial da Saude?,This document does not contain information about the World Health Organization's proposed goals.,-,0.446222,This document does not contain information about the goals proposed by the World Health Organization.,-,0.419588
3,4. Que fatores influenciam as doenças que as pessoas contraem?,"Based on the provided text, several factors influence the diseases people contract. These include lifestyle choices (diet, exercise, alcohol cons...",-,0.379653,"Based on the provided text, several factors influence the diseases people contract. These include:\n\n* **Age:** Infancy and old age are mention...",-,0.414291
4,5. O que é o modelo da vigilância da saude?,"Based on the provided text, I cannot answer your question. The text discusses the relationship between health and social factors, the history of ...",-,0.310083,"Based on the provided text, the model of health surveillance is described as a recent and expanding field in Brazil. It emerged from the decentra...",-,0.409978



--- Tabela de comparação salva com sucesso em: ---
/content/drive/MyDrive/RAG/rag_comparison_results.csv

Processo de Comparação RAG concluído.
